<a href="https://colab.research.google.com/github/rodrichie/NLP-FakeNewsClassifierUsingLSTM/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fake News Classifier Using LSTM

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/content/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [6]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [7]:
###Drop Nan values
df=df.dropna()

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
# Get the independent features
X=df.drop('label' , axis=1)

In [10]:
# Get the Dependent features
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [21]:
import tensorflow as tf

In [22]:
tf.__version__

'2.17.1'

In [23]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [24]:
### vocabulary size
voc_size=5000

### Onehot Representation

In [25]:
messages=X.copy()

In [26]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [27]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [28]:
#messages.reset_index(inplace=True)

In [29]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [30]:
import nltk
import re
from nltk.corpus import stopwords

In [31]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [33]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [34]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [35]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [36]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[4427, 818, 3591, 335, 121, 2350, 2189, 2520, 1086, 3385],
 [2424, 3945, 3629, 1520, 2757, 700, 4181],
 [2934, 4610, 3024, 301],
 [1768, 525, 572, 3651, 2204, 4963],
 [2007, 2757, 568, 1013, 3750, 1391, 2757, 1519, 438, 4239],
 [2446,
  545,
  916,
  4304,
  3117,
  783,
  1786,
  520,
  227,
  4168,
  2366,
  2395,
  2079,
  4789,
  4181],
 [3422, 4435, 1322, 2943, 2258, 1249, 3787, 1361, 99, 2351, 4082],
 [1826, 769, 903, 3923, 3560, 4635, 783, 2667, 99, 2351, 4082],
 [4452, 2338, 1695, 3776, 4516, 1852, 3943, 2191, 783, 705],
 [4985, 1109, 4333, 315, 4984, 4986, 4496, 1461],
 [3799, 4592, 4073, 3892, 4083, 3551, 4997, 2387, 2302, 2851, 3226],
 [3651, 1221, 121, 1852, 783, 3560],
 [3124, 1720, 2271, 701, 3360, 1527, 686, 115, 2579],
 [3858, 1579, 4642, 3846, 4224, 4301, 713, 99, 2351, 4082],
 [227, 3741, 1814, 656, 1591, 99, 2351, 4082],
 [1058, 792, 2533, 3843, 860, 3181, 2385, 2963, 599, 13],
 [3532, 559, 3945],
 [3252, 4967, 695, 2507, 783, 3118, 4246, 4181],
 [4518, 1496, 3629, 

In [37]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [38]:
onehot_repr[1]

[2424, 3945, 3629, 1520, 2757, 700, 4181]

### Embedding Representation

In [39]:
sent_length= 20
embedded_docs= pad_sequences(onehot_repr, padding='post', maxlen=sent_length)
print(embedded_docs)

[[4427  818 3591 ...    0    0    0]
 [2424 3945 3629 ...    0    0    0]
 [2934 4610 3024 ...    0    0    0]
 ...
 [4071 1640  458 ...    0    0    0]
 [2823 3560 2927 ...    0    0    0]
 [1289  374 4190 ...    0    0    0]]


In [40]:
embedded_docs[1]

array([2424, 3945, 3629, 1520, 2757,  700, 4181,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [41]:
embedded_docs[0]

array([4427,  818, 3591,  335,  121, 2350, 2189, 2520, 1086, 3385,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)